# TLJ Topic Modeling - NMF

In [5]:
import pandas as pd
import numpy as np
import re 
from pprint import pprint

import pickle


# import string


# from sklearn.feature_extraction.text import CountVectorizer

# from textblob import TextBlob

import nltk, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.util import *

# from sklearn.metrics import accuracy_score, confusion_matrix

# from confusion import print_confusion_matrix

# import random

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from pprint import pprint


In [6]:
# Import tlj df pickle
tlj_all = pickle.load(open('tlj_v2.pickle', 'rb'))

In [7]:
# Create NLTK stopwords
stop_words = stopwords.words('english')

In [9]:
# Create DataFrame where that only has negative sentiment reviews
tlj = tlj_all.loc[tlj_all['google_sentiment']=='negative']

### Remove Stop Words and Lemmatization Functions

In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return[[word for word in doc if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Preprocess with TF-IDF

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
tlj_tfidf = tfidf.fit_transform(list(tlj['Reviews']))

## Find Topics with NMF Model

In [16]:
# Function to display topics
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Topic Count: 5

In [14]:
# Start with 5 Topics to see what the results are
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [15]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", ],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aback,abaloth,abandon,abandoned,abandoning,abandonment,abandonned,abandons,abborent,...,zooming,zooms,zornord,zu,zucchini,zucker,état,être,últimos,über
component_1,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,...,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.001,0.0,0.000,0.013,0.017,0.003,0.003,0.001,0.003,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001
component_3,0.000,0.0,0.001,0.000,0.000,0.002,0.000,0.000,0.002,0.000,...,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.0,0.000,0.000,0.000,0.003,0.000,0.000,0.003,0.000,...,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.004,0.000
component_5,0.000,0.0,0.000,0.001,0.007,0.000,0.000,0.000,0.000,0.001,...,0.003,0.000,0.000,0.001,0.001,0.002,0.001,0.001,0.000,0.001


In [21]:
topics_nmf_5=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_5


Topic  0
movie, bad, just, like, don, really, good, people, worst, think, reviews, didn, time, say, movies, watch, make, watching, things, want

Topic  1
luke, rey, kylo, force, snoke, jedi, ren, leia, finn, like, order, just, character, rose, did, resistance, scene, space, skywalker, poe

Topic  2
film, films, like, just, really, think, don, good, going, bad, people, reviews, say, ve, feel, time, lot, seen, review, didn

Topic  3
star, wars, disney, fan, worst, jedi, fans, movies, universe, johnson, lucas, franchise, rian, watch, ruined, saga, money, original, films, like

Topic  4
story, characters, new, plot, character, trilogy, episode, good, original, holes, johnson, development, rian, make, franchise, old, movies, sw, disney, line


### Topic Count: 10

In [22]:
# Start with 5 Topics to see what the results are
nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [23]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", "component_6", "component_7", "component_8", "component_9", "component_10"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aback,abaloth,abandon,abandoned,abandoning,abandonment,abandonned,abandons,abborent,...,zooming,zooms,zornord,zu,zucchini,zucker,état,être,últimos,über
component_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.002,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.001,0.000,0.000,0.005,0.017,0.001,0.004,0.002,0.003,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.002,0.000,...,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.004,0.000
component_5,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,...,0.002,0.000,0.000,0.000,0.001,0.002,0.001,0.001,0.001,0.000
component_6,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000
component_7,0.000,0.000,0.001,0.018,0.007,0.003,0.000,0.000,0.001,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.002
component_8,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.005,0.000,...,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.002,...,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.001,0.000,0.001,0.000,0.002,0.000,0.000,0.001,0.003,...,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.001


In [24]:
topics_nmf_10=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_10


Topic  0
like, just, don, really, people, didn, think, felt, know, good, going, feel, say, make, things, did, time, want, wasn, lot

Topic  1
luke, rey, kylo, force, jedi, ren, snoke, skywalker, character, leia, yoda, training, dark, lightsaber, ben, vader, kill, finn, awakens, powerful

Topic  2
film, films, character, jedi, good, going, awakens, previous, seen, action, franchise, ve, lot, audience, characters, feel, review, watching, scene, fans

Topic  3
star, wars, fan, disney, jedi, movies, universe, fans, watch, films, original, lucas, new, disappointed, saga, series, like, force, franchise, ruined

Topic  4
characters, story, plot, new, character, holes, trilogy, development, good, original, movies, line, old, franchise, main, scenes, effects, make, great, sw

Topic  5
movie, movies, watching, boring, point, scenes, humor, great, previous, scene, jokes, wrong, watch, sad, entire, minutes, makes, good, stars, plot

Topic  6
space, ship, order, finn, ships, rose, scene, resistanc

- Interestingly, this Topic 9 is the most decipherable, so creating more topics might help. Let's try 15 next. 
- Topic 9 seems to be the "Franchise Stewardship" topic, where people are talking about directors, producers, and studios, along with assorted negative words. 

### Topic Count: 15

In [25]:
# Start with 5 Topics to see what the results are
nmf_model = NMF(15)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [26]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", 
                                    "component_6", "component_7", "component_8", "component_9", "component_10",
                                    "component_11", "component_12", "component_13", "component_14", "component_15"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aback,abaloth,abandon,abandoned,abandoning,abandonment,abandonned,abandons,abborent,...,zooming,zooms,zornord,zu,zucchini,zucker,état,être,últimos,über
component_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.001,0.000,0.000,0.004,0.014,0.001,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.001,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.000,0.000,0.000,0.000,0.015,0.000,0.001,0.000,0.003,0.000,...,0.001,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.003,0.001
component_6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_7,0.000,0.000,0.001,0.021,0.009,0.003,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001
component_8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,...,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.006,0.001
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.002,...,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.001,0.000,0.002,0.000,0.003,0.000,0.000,0.000,0.003,...,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.000,0.000


In [27]:
topics_nmf_15=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_15


Topic  0
like, really, didn, don, felt, think, people, feel, things, lot, good, did, know, wasn, liked, wanted, make, feels, hate, scenes

Topic  1
luke, rey, kylo, force, jedi, ren, snoke, skywalker, character, leia, training, yoda, dark, lightsaber, ben, kill, finn, vader, killed, powerful

Topic  2
film, films, character, going, previous, franchise, awakens, time, seen, audience, jedi, characters, ve, review, good, reviews, watching, action, set, left

Topic  3
star, wars, fan, universe, movies, watch, jedi, disappointed, fans, films, like, series, watched, saga, rogue, ruined, terrible, feel, loved, huge

Topic  4
new, trilogy, original, characters, old, jedi, force, movies, awakens, sw, fans, empire, hope, universe, franchise, generation, tlj, ones, far, prequels

Topic  5
movie, movies, watching, point, boring, time, reviews, previous, watch, sad, wrong, want, great, humor, stars, jokes, makes, good, left, watched

Topic  6
space, ship, order, finn, resistance, ships, rose, leia

- The more topics that we add the more sense they start to make, interestingly. Let's go to 20!

### Topic Count: 20

In [28]:
# Start with 5 Topics to see what the results are
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [29]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", 
                                    "component_6", "component_7", "component_8", "component_9", "component_10",
                                    "component_11", "component_12", "component_13", "component_14", "component_15",
                                    "component_16", "component_17", "component_18", "component_19", "component_20"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aback,abaloth,abandon,abandoned,abandoning,abandonment,abandonned,abandons,abborent,...,zooming,zooms,zornord,zu,zucchini,zucker,état,être,últimos,über
component_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.001,0.000,0.000,0.004,0.010,0.001,0.002,0.002,0.001,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.001,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.000,0.000,0.000,0.003,0.004,0.000,0.000,0.000,0.000,0.000,...,0.001,0.000,0.000,0.001,0.001,0.002,0.000,0.000,0.003,0.001
component_6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_7,0.000,0.000,0.001,0.024,0.012,0.003,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001
component_8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,...,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.007,0.000
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.002,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.001,0.000,0.002,0.000,0.003,0.000,0.000,0.000,0.003,...,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.000,0.000


In [30]:
topics_nmf_20=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_20


Topic  0
like, really, felt, didn, feel, scenes, lot, good, things, scene, great, feels, did, liked, bit, wasn, little, action, moments, way

Topic  1
luke, rey, kylo, ren, snoke, character, skywalker, jedi, leia, yoda, finn, lightsaber, ben, dark, training, kill, did, killed, vader, force

Topic  2
film, films, character, franchise, previous, characters, going, audience, seen, ve, action, good, lot, moments, script, direction, left, feel, set, watching

Topic  3
star, wars, fan, universe, movies, watch, disappointed, fans, films, series, jedi, watched, like, saga, rogue, original, ruined, feel, loved, huge

Topic  4
new, characters, trilogy, original, story, old, movies, good, generation, ones, fans, empire, make, universe, hope, franchise, care, far, heroes, end

Topic  5
movie, movies, watching, point, previous, sad, left, stars, wrong, makes, great, boring, entire, humor, watched, jokes, theater, things, minutes, series

Topic  6
space, ship, order, ships, resistance, finn, leia, 

- We don't seem to be getting many clearer topics now, though it's not much more vague than 15. We'll stop here. 

### Bigrams and Trigrams
Let's see if Bigrams and Trigrams get us any clearer topics

#### Bigrams

In [50]:
# Bigram TF-IDF
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tlj_tfidf = tfidf.fit_transform(list(tlj['Reviews']))

In [38]:
# 15 Topics to see what the results are
nmf_model = NMF(15)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [39]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", 
                                    "component_6", "component_7", "component_8", "component_9", "component_10",
                                    "component_11", "component_12", "component_13", "component_14", "component_15"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aa long,aback,aback wants,abaloth,abaloth telling,abandon,abandon altogether,abandon approach,abandon base,...,zucker films,état,état apparently,être,être luke,últimos,últimos jedi,über,über alles,über powered
component_1,0.002,0.002,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,...,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.000,0.000
component_3,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.001,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000
component_6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_7,0.000,0.000,0.000,0.000,0.001,0.001,0.017,0.000,0.002,0.001,...,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.001
component_8,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,...,0.001,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.000,0.000
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [40]:
topics_nmf_15_2ngram=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_15_2ngram


Topic  0
kylo, rey, ren, snoke, character, kylo ren, finn, luke, rose, force, poe, leia, scene, great, really, resistance, did, tfa, phasma, rey kylo

Topic  1
star, star wars, wars, disney, fan, wars movie, wars fan, universe, movies, wars movies, wars universe, fans, wars fans, watch, lucas, jedi, films, disappointed, wars film, like

Topic  2
film, wars film, films, good, character, characters, reviews, jedi, film just, franchise, going, star wars, star, wars, seen, say, ve, previous, audience, think

Topic  3
movie, wars movie, good, movie just, movies, reviews, great, good movie, watch, movie movie, just, boring, watching, scenes, people, make, point, movie good, previous, things

Topic  4
johnson, rian, rian johnson, abrams, director, kennedy, kathleen, disney, kathleen kennedy, hamill, mark, character, mark hamill, did, disney rian, episode, fans, trilogy, director rian, skywalker

Topic  5
worst, worst star, worst movie, movie worst, seen, wars movie, ve seen, movie, ve, movie

In [51]:
# 20 Topics to see what the results are
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [53]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", 
                                    "component_6", "component_7", "component_8", "component_9", "component_10",
                                    "component_11", "component_12", "component_13", "component_14", "component_15",
                                    "component_16", "component_17", "component_18", "component_19", "component_20"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aa long,aback,aback wants,abaloth,abaloth telling,abandon,abandon altogether,abandon approach,abandon base,...,zucker films,état,état apparently,être,être luke,últimos,últimos jedi,über,über alles,über powered
component_1,0.002,0.002,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.000,0.000
component_3,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.001,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.000
component_6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_7,0.000,0.000,0.000,0.000,0.001,0.001,0.019,0.000,0.002,0.002,...,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.001
component_8,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [61]:
display_topics(nmf_model, tfidf.get_feature_names(), 20)


Topic  0
kylo, rey, ren, snoke, kylo ren, luke, character, finn, rose, leia, poe, force, scene, really, great, did, resistance, lightsaber, just, parents

Topic  1
star, star wars, wars, disney, fan, wars fan, wars movie, universe, movies, wars universe, wars movies, fans, wars fans, lucas, jedi, watch, films, franchise, wars film, saga

Topic  2
film, wars film, films, character, good, film just, characters, franchise, jedi, previous, audience, seen, action, lot, ve, wars films, star wars, wars, moments, film star

Topic  3
movie, wars movie, good, movie just, movie movie, great, good movie, watching, boring, movies, scenes, movie good, plot, scene, previous, humor, watch movie, movie star, point, watch

Topic  4
johnson, rian, rian johnson, abrams, kennedy, kathleen, director, kathleen kennedy, disney, hamill, disney rian, mark, trilogy, mark hamill, character, did, fans, director rian, hate rian, skywalker

Topic  5
worst, worst star, worst movie, movie worst, seen, wars movie, ve 

- This is by far the clearest set of topics I've seen up until this point. 

In [55]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(nmf_model, tlj_tfidf, tfidf, mds='tsne')
panel

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1     -14.232074  12.571416       1        1  11.853135
3      -5.423209   4.511973       2        1   7.953353
2      -7.887531  11.201767       3        1   7.638481
14    -18.376606  17.047258       4        1   7.355565
0     -17.992231  27.066696       5        1   7.230712
7      -1.458841  10.014186       6        1   6.623404
6     -11.190905  17.008482       7        1   6.509524
10    -10.360239  28.095577       8        1   6.489073
17      1.334007  16.388165       9        1   6.192294
9     -24.726755  14.926590      10        1   4.152952
4     -11.359372   0.734612      11        1   4.011011
19    -18.231012   3.675800      12        1   4.006678
8     -14.630592  21.798880      13        1   3.634793
13    -22.971268  21.794777      14        1   3.492797
16    -19.362188  10.666140      15        1   2.864869
18     -5.314947  16.446205      16        1   2.483333
12    -24.775831   7.214049      17        1   2.150171
11    -12.454656   6.943321      18        1   2.035486
5      -1.881109  23.534061      19        1   1.828292
15     -8.225608  21.933136      20        1   1.494077, topic_info=       Category        Freq                   Term       Total  loglift  \
term                                                                      
282934  Default  135.000000              star wars  135.000000  30.0000   
326254  Default  135.000000                   wars  135.000000  29.0000   
282692  Default  138.000000                   star  138.000000  28.0000   
195666  Default  121.000000                  movie  121.000000  27.0000   
18408   Default   45.000000                    bad   45.000000  26.0000   
104397  Default   59.000000                   film   59.000000  25.0000   
336625  Default   24.000000                  worst   24.000000  24.0000   
88274   Default   26.000000                episode   26.000000  23.0000   
179822  Default   46.000000                   luke   46.000000  22.0000   
327129  Default   39.000000             wars movie   39.000000  21.0000   
170395  Default   50.000000                   like   50.000000  20.0000   
293868  Default   17.000000                     sw   17.000000  19.0000   
221597  Default   33.000000                   plot   33.000000  18.0000   
336841  Default   16.000000             worst star   16.000000  17.0000   
136750  Default   18.000000                  holes   18.000000  16.0000   
152058  Default   15.000000                    jar   15.000000  15.0000   
221935  Default   17.000000             plot holes   17.000000  14.0000   
328210  Default   15.000000                  waste   15.000000  13.0000   
154148  Default   19.000000                johnson   19.000000  12.0000   
111781  Default   30.000000                  force   30.000000  11.0000   
250152  Default   18.000000                   rian   18.000000  10.0000   
328304  Default   14.000000             waste time   14.000000   9.0000   
250241  Default   16.000000           rian johnson   16.000000   8.0000   
303737  Default   29.000000                   time   29.000000   7.0000   
162173  Default   21.000000                   kylo   21.000000   6.0000   
111838  Default   15.000000          force awakens   15.000000   5.0000   
16096   Default   16.000000                awakens   16.000000   4.0000   
194287  Default   17.000000                  money   17.000000   3.0000   
249006  Default   27.000000                    rey   27.000000   2.0000   
152256  Default   39.000000                   jedi   39.000000   1.0000   
...         ...         ...                    ...         ...      ...   
56188   Topic20    0.482092             crawl mean    0.482092   4.2037   
137172  Topic20    0.482092  hollywood blockbuster    0.482092   4.2037   
116551  Topic20    0.482092                fun let    0.482092   4.2037   
292654  Topic20    0.4820

### Trigrams
Let's try the trigrams and see what happens

In [44]:
# Trigram TF-IDF
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tlj_tfidf = tfidf.fit_transform(list(tlj['Reviews']))

In [45]:
# We'll start with 20 Topics to see what the results are
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(tlj_tfidf)

In [46]:
topic_words = pd.DataFrame(nmf_model.components_.round(3),
                           index = ["component_1", "component_2", "component_3", "component_4", "component_5", 
                                    "component_6", "component_7", "component_8", "component_9", "component_10",
                                    "component_11", "component_12", "component_13", "component_14", "component_15",
                                    "component_16", "component_17", "component_18", "component_19", "component_20"],
                           columns = tfidf.get_feature_names())
topic_words

,aa,aa long,aa long time,aback,aback wants,aback wants applaud,abaloth,abaloth telling,abaloth telling character,abandon,...,être luke,être luke train,últimos,últimos jedi,últimos jedi star,über,über alles,über alles crowd,über powered,über powered mary
component_1,0.002,0.002,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.003,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.002,0.002,0.002,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.001,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.002,0.002,0.002,0.001,0.000,0.000,0.000,0.000
component_6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_7,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000
component_8,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.001,0.020,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.001
component_9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_10,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [48]:
topics_nmf_20_3ngram=display_topics(nmf_model, tfidf.get_feature_names(), 20)
topics_nmf_20_3ngram


Topic  0
kylo, rey, snoke, ren, luke, character, kylo ren, finn, rose, force, scene, leia, really, poe, just, did, great, like, didn, tfa

Topic  1
star, star wars, wars, disney, universe, movies, wars movie, star wars movie, star wars universe, wars universe, star wars movies, wars movies, fans, jedi, wars fans, star wars fans, lucas, franchise, films, saga

Topic  2
film, wars film, star wars film, films, wars, star wars, star, characters, good, character, jedi, film just, franchise, action, previous, ve, lot, scenes, moments, seen

Topic  3
movie, wars movie, star wars movie, good, movie just, scenes, just, great, movie movie, boring, watching, plot, movies, good movie, character, scene, movie good, things, humor, previous

Topic  4
new, characters, trilogy, story, original, sw, old, new characters, original trilogy, movies, episode, character, good, tlj, franchise, plot, universe, tfa, effects, new trilogy

Topic  5
corporate, assembly line, crass, assembly, rip offs, cinematic, h

## Visualize

In [49]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(nmf_model, tlj_tfidf, tfidf, mds='tsne')
panel

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1     -165.553329   99.204941       1        1  11.058704
3       94.641426  165.820923       2        1   8.790390
0       33.544094  -52.629860       3        1   8.507209
2       54.319130   92.955696       4        1   8.478070
9      -89.190826  151.854935       5        1   8.359503
12     143.089523   79.177948       6        1   7.394135
4      -41.857140   -4.182203       7        1   7.193760
7     -112.017021  -35.534298       8        1   6.018951
10     -39.662510  -82.647263       9        1   4.913771
6      -29.377092   88.372612      10        1   4.496067
16     114.868149  -89.432938      11        1   4.450811
17     -98.275307   49.568611      12        1   4.166208
8       20.744429   26.469812      13        1   4.092032
15      -0.729766  172.592453      14        1   2.784471
14      40.183167 -139.336411      15        1   1.984343
11    -137.691406 -113.602051      16        1   1.922062
13     172.558273  -14.518068      17        1   1.841877
19      92.148346    4.920871      18        1   1.709880
5     -186.153900    1.827115      19        1   1.036240
18     -50.546890 -164.748978      20        1   0.801517, topic_info=       Category       Freq                   Term      Total  loglift  logprob
term                                                                          
40231   Default  36.000000                    bad  36.000000  30.0000  30.0000
668491  Default  67.000000              star wars  67.000000  29.0000  29.0000
769252  Default  68.000000                   wars  68.000000  28.0000  28.0000
667741  Default  69.000000                   star  69.000000  27.0000  27.0000
460227  Default  75.000000                  movie  75.000000  26.0000  26.0000
731127  Default  15.000000                  trash  15.000000  25.0000  25.0000
237242  Default  40.000000                   film  40.000000  24.0000  24.0000
797195  Default  16.000000                  worst  16.000000  23.0000  23.0000
773430  Default  21.000000             wars movie  21.000000  22.0000  22.0000
669358  Default  21.000000        star wars movie  21.000000  21.0000  21.0000
797788  Default  11.000000             worst star  11.000000  20.0000  20.0000
797791  Default  11.000000        worst star wars  11.000000  19.0000  19.0000
526583  Default  12.000000             plot holes  12.000000  18.0000  18.0000
315607  Default  13.000000                  holes  13.000000  17.0000  17.0000
421241  Default  31.000000                   luke  31.000000  16.0000  16.0000
348330  Default   9.000000                    jar   9.000000  15.0000  15.0000
777722  Default  10.000000                  waste  10.000000  14.0000  14.0000
525540  Default  24.000000                   plot  24.000000  13.0000  13.0000
41759   Default  12.000000              bad movie  12.000000  12.0000  12.0000
684890  Default  12.000000                 stupid  12.000000  11.0000  11.0000
354674  Default  14.000000                johnson  14.000000  10.0000  10.0000
718197  Default  19.000000                   time  19.000000   9.0000   9.0000
592699  Default  12.000000                   rian  12.000000   8.0000   8.0000
222008  Default  14.000000                    fan  14.000000   7.0000   7.0000
777959  Default   8.000000             waste time   8.000000   6.0000   6.0000
396796  Default  32.000000                   like  32.000000   5.0000   5.0000
592900  Default  10.000000           rian johnson  10.000000   4.0000   4.0000
457375  Default  11.000000                  money  11.000000   3.0000   3.0000
770709  Default   8.000000               wars fan   8.000000   2.0000   2.0000
668931  Default   8.000000          star wars fan   8.000000   1.0000   1.0000
...         ...        ...                    ...        ...      ...      ...
121504  Topic20   0.252260           corporatized   0.314877   4.6047  -7.

In [ ]:
###